In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
num_features = 784 # data features (img shape: 28*28).

# Training parameters.
learning_rate = 0.001
training_steps = 3000
batch_size = 256
display_step = 100

# Network parameters.
n_hidden_1 = 128 # 1st layer number of neurons.
n_hidden_2 = 256 # 2nd layer number of neurons.

In [3]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Flatten images to 1-D vector of 784 features (28*28).
x_train, x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

In [4]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)

In [5]:
random_normal = tf.initializers.RandomNormal()

weights = {
    'h1' : tf.Variable( random_normal( [num_features, n_hidden_1] ) ),
    'h2' : tf.Variable( random_normal( [n_hidden_1, n_hidden_2] ) ),
    'out' : tf.Variable( random_normal( [n_hidden_2, num_classes] ) )
}

biases = {
    'b1' : tf.Variable( random_normal( [n_hidden_1] ) ),
    'b2' : tf.Variable( random_normal( [n_hidden_2] ) ),
    'b_out' : tf.Variable( random_normal( [num_classes] ) )
}

In [16]:
def neural_net(x):

    layer_1 = tf.add( tf.matmul( x, weights['h1'] ), biases['b1'] )
    layer_1 = tf.nn.sigmoid(layer_1)

    layer_2 = tf.add( tf.matmul( layer_1, weights['h2'] ), biases['b2'] )
    layer_2 = tf.nn.sigmoid( layer_2 )

    out = tf.add( tf.matmul( layer_2, weights['out'] ), biases['b_out'] )

    return tf.nn.softmax(out)

In [17]:
# Cross-Entropy loss function.
def cross_entropy(y_pred, y_true):
    # Encode label to a one hot vector.
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction values to avoid log(0) error.
    y_pred = tf.clip_by_value(y_pred, 1e-9, 1.)
    # Compute cross-entropy.
    return tf.reduce_mean(-tf.reduce_sum(y_true * tf.math.log(y_pred)))

# Accuracy metric.
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e. argmax).
    correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32), axis=-1)

# Stochastic gradient descent optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

In [22]:
def run_optimization(x, y):

    with tf.GradientTape() as tape:

        pred = neural_net(x)

        loss = cross_entropy(pred, y)

    trainable_variables = [*weights.values(), *biases.values()]
    
    gradients = tape.gradient(loss, trainable_variables)

    optimizer.apply_gradients(zip(gradients, trainable_variables))

In [21]:
for step, (batch_x, batch_y) in enumerate(train_data, 1):

    run_optimization(batch_x, batch_y)

    if step % 50 == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)

        print("step: %i, loss: %f, accuracy: %f" % (step, loss, acc))


AttributeError: 'dict_values' object has no attribute 'device'